In [3]:
!pip install flask
!pip install sympy
!pip install flask_cors
!pip install sympy
!pip install mysql-connector-python
!pip install mediapipe opencv-python


In [2]:
from urllib import parse
from unicodedata import category
from flask import Flask,request,redirect,render_template
from flask_cors import CORS,cross_origin
from fractions import Fraction
from sympy import re

#--------------------------------------------------
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
from sklearn.preprocessing import StandardScaler
from cv2 import destroyAllWindows
import pandas as pd
from PIL import ImageFont, ImageDraw, Image
from tensorflow.keras.models import load_model

#---------------------------------------------------------
import mysql.connector
from ast import AsyncFunctionDef
from audioop import cross
from unittest import result



AttributeError: module 'mediapipe.python.solutions' has no attribute 'drawing_styles'

In [2]:
def insert(val):
    
    conn = mysql.connector.connect(host='project-db-stu.ddns.net', user='smhrdf004',
    password='smhrdf004', db='smhrdf004', port=3307, use_unicode=True, charset='utf8')
    cursor=conn.cursor()
    
    for i in range(len(val)):
        sql=f"INSERT INTO feedback (nickname, ex_name, accuracy, feed_date, Feed_con) values ('admin', '랫 풀 다운', '02.00', date_add(sysdate(), INTERVAL 1 DAY), '{val[i]}')"
        cursor.execute(sql)
        conn.commit()
       
    cursor.close()
    conn.close()


In [ ]:
cursor=conn.cursor()
query="INSERT INTO feedback (nickname, ex_name, accuracy, feed_date, Feed_con) values ('admin', '랫 풀 다운', 00.01, date_add(sysdate(), INTERVAL 1 DAY), 'test')"

cursor.execute(query)
row = cursor.fetchone()
print(row)
conn.commit
cursor.close()
conn.close()
print('MySQL Connection Close')

In [6]:
from enum import auto
from multiprocessing.sharedctypes import Value
from turtle import width

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 


from enum import auto
from turtle import width

def lat():

    cap = cv2.VideoCapture(0)
    # Curl counter variables
    counter = 0 

    result = []
    stage = 'up'
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
                
                
                shoulder_L = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y] # 왼쪽
                shoulder_R = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y] # 오른쪽
                elbow_L = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y] # 왼쪽
                elbow_R = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y] # 오른쪽
                wrist_L = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y] # 왼쪽
                wrist_R = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y] # 오른쪽
                hip_L = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y] # 왼쪽
                hip_R = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y] # 오른쪽
                
                
                
                
                
                
                # Calculate angle
                angle = calculate_angle(shoulder, elbow, wrist)
                
                # Visualize angle
                cv2.putText(image, str(angle), 
                            tuple(np.multiply(elbow, [1920, 1080]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                
                # Curl counter logic
                if stage == 'up' :
                    if angle > 160 :
                        stage = "down"
                        s = 1
                        if counter >= 1:
                            twoShape = []
                            oneShape = []
                            for num in ['shoulder_L','shoulder_R','elbow_L','elbow_R','wrist_L','wrist_R','hip_L','hip_R']:
                                oneShape += list(tuple(np.multiply(eval(f'{num}'), [1920, 1080]).astype(int)))
                            twoShape += [oneShape]
                            result += [twoShape]
                                
                if angle < 30 and stage =='down':
                    stage="up"
                    if s == 1 :
                        if counter == 5 :
                            cv2.waitKey(3000)
                            break
                    counter +=1
                    
                
                        
            except:
                pass
            
            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
            
            # Rep data
            cv2.putText(image, 'REPS', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Stage data
            cv2.putText(image, 'STAGE', (65,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (60,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )               
            
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

        np.array(result).shape
        model = load_model('./model2.h5')
        a = np.array(result)
        b = model.predict(a)
        c = np.argmax(b, axis=1)
        print(c)
        val=[]
        acc = 0
        for i in range(len(c)):
            if c[i]==0: 
                acc+=20
                val.append("정자세입니다.")
            elif c[i]==1:
                val.append("기구 당김이 부족합니다.")
            elif c[i]==2:           
                val.append("상체가 불안정하며, 기구 당김이 부족합니다.")
            elif c[i]==3:
                val.append("축 평행이 불량하며, 기구 당김이 부족합니다.")
         
        print(acc)
        
        for j in range(len(val)): 
            print(val[j])
        
        insert(val)
        


In [8]:
lat()

[12 12  0 12 12]
20
상체가 자세 불안정하며, 기구 당김이 부족합니다.
상체가 자세 불안정하며, 기구 당김이 부족합니다.
정자세입니다.
상체가 자세 불안정하며, 기구 당김이 부족합니다.
상체가 자세 불안정하며, 기구 당김이 부족합니다.


In [84]:







app=Flask(__name__)  #flask 서버 객체 생성

@cross_origin
@app.route("/")

def index():
    url = "https://www.naver.com/"

    return redirect(url)


@cross_origin
@app.route("/naver", methods=["GET", "POST"])
def test():
    if request.method=='POST':
        
        url="http://localhost:8085/flask"
        return redirect(url)
    else:
        lat()
        
        url_2 ="http://localhost:8085/flask"

        return redirect(url_2)
      

# main함수의 시작을 의미 
# 비유하자면 자바의 main(String[] args)메소드와 같은 역할


if __name__ == '__main__':
    # 웹서버주소(host)와 포트번호(port)를 지정하여 웹 서버 실행
    app.run(host='127.0.0.1', port='5002') 

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)


[0 0 0 0 0]
정자세입니다.
정자세입니다.
정자세입니다.
정자세입니다.
정자세입니다.


127.0.0.1 - - [09/Oct/2022 13:47:12] "GET /naver HTTP/1.1" 302 -


[ 0  0  0  0 12]
정자세입니다.
정자세입니다.
정자세입니다.
정자세입니다.
상체가 자세 불안정하며, 기구 당김이 부족합니다.


127.0.0.1 - - [09/Oct/2022 13:48:03] "GET /naver HTTP/1.1" 302 -


In [ ]:
import mysql.connector
conn = mysql.connector.connect(host='project-db-stu.ddns.net', user='smhrdf004', 
password='smhrdf004', db='smhrdf004', port=3307, use_unicode=True, charset='utf8')

cursor=conn.cursor()


cursor.execute("INSERT INTO feedback (nickname, ex_name, accuracy, feed_date, Feed_con) values ('admin', '랫 풀 다운', 00.01, date_add(sysdate(), INTERVAL 1 DAY), 'test')")
conn.commit()
row = cursor.fetchone()
cursor.close()
conn.close()
print('MySQL Connection Close')

def insert():
    conn = mysql.connector.connect(host='project-db-stu.ddns.net', user='smhrdf004',
    password='smhrdf004', db='smhrdf004', port=3307, use_unicode=True, charset='utf8')
    cursor=conn.cursor()
    
    sql="INSERT INTO feedback (nickname, ex_name, accuracy, feed_date, Feed_con) values (%s, %s, %f, date_add(sysdate(), INTERVAL 1 DAY), %s)"
    result = (res.nickname, res.ex_name, res.accuracy, res.feed_con)
    cursor.execute(sql, result)
    conn.commit()
    row = cursor.fetchone()
    cursor.close()
    conn.close()

    return print("성공")
